# Whittier HQ transit stops

In [1]:
import pandas as pd
import geopandas as gpd
import intake
import shapely

catalog = intake.open_catalog("*.yml")

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(
/tmp/ipykernel_393/3791979869.py:2: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  im

In [2]:
df = catalog.hqta_points.read()

In [3]:
df.columns

Index(['agency_name_primary', 'hqta_type', 'stop_id', 'route_id',
       'hqta_details', 'agency_name_secondary', 'base64_url_primary',
       'base64_url_secondary', 'geometry'],
      dtype='object')

In [4]:
agencies = ["LA Metro", "Foothill", "Montebello", "OCTA", "LADPW"]
df = df[(df.agency_name_primary.isin(agencies))]

In [5]:

centroid = (-118.030246, 33.965747)

whittier_centroid = gpd.GeoSeries(
    shapely.geometry.Point(centroid), crs="EPSG:4326"
).to_frame(name="geometry")

In [6]:
whittier_buffered = whittier_centroid.assign(
    geometry = whittier_centroid.geometry.to_crs("EPSG:2229").buffer(
        5280*5)
)

In [7]:
df2 = gpd.sjoin(
    df.to_crs("EPSG:2229"),
    whittier_buffered,
    how = "inner",
    predicate = "intersects"
).drop(columns = "index_right")

In [52]:
df2.hqta_type.value_counts()

hq_corridor_bus    309
major_stop_bus      90
Name: hqta_type, dtype: int64

In [8]:
df2.hqta_details.value_counts()

stop_along_hq_bus_corridor_single_operator       309
intersection_2_bus_routes_same_operator           56
intersection_2_bus_routes_different_operators     34
Name: hqta_details, dtype: int64

In [9]:
df3 = df2[(df2.hqta_details.isin(
    ["intersection_2_bus_routes_same_operator", 
    "intersection_2_bus_routes_different_operator"]))]

In [45]:
df3

,agency_name_primary,hqta_type,stop_id,route_id,hqta_details,agency_name_secondary,base64_url_primary,base64_url_secondary,geometry
8643,Foothill,major_stop_bus,1398,281,intersection_2_bus_routes_same_operator,Foothill,aHR0cHM6Ly9mb290aGlsbHRyYW5zaXQucmlkZXJhbGVydH...,aHR0cHM6Ly9mb290aGlsbHRyYW5zaXQucmlkZXJhbGVydH...,POINT (6570548.556 1827200.287)
8644,Foothill,major_stop_bus,1399,281,intersection_2_bus_routes_same_operator,Foothill,aHR0cHM6Ly9mb290aGlsbHRyYW5zaXQucmlkZXJhbGVydH...,aHR0cHM6Ly9mb290aGlsbHRyYW5zaXQucmlkZXJhbGVydH...,POINT (6570681.319 1827115.168)
8645,Foothill,major_stop_bus,1406,282,intersection_2_bus_routes_same_operator,Foothill,aHR0cHM6Ly9mb290aGlsbHRyYW5zaXQucmlkZXJhbGVydH...,aHR0cHM6Ly9mb290aGlsbHRyYW5zaXQucmlkZXJhbGVydH...,POINT (6567803.855 1828438.394)
8646,Foothill,major_stop_bus,1407,281,intersection_2_bus_routes_same_operator,Foothill,aHR0cHM6Ly9mb290aGlsbHRyYW5zaXQucmlkZXJhbGVydH...,aHR0cHM6Ly9mb290aGlsbHRyYW5zaXQucmlkZXJhbGVydH...,POINT (6567721.428 1828435.103)
8647,Foothill,major_stop_bus,1411,281,intersection_2_bus_routes_same_operator,Foothill,aHR0cHM6Ly9mb290aGlsbHRyYW5zaXQucmlkZXJhbGVydH...,aHR0cHM6Ly9mb290aGlsbHRyYW5zaXQucmlkZXJhbGVydH...,POINT (6569082.234 1827816.011)
8648,Foothill,major_stop_bus,1412,282,intersection_2_bus_routes_same_operator,Foothill,aHR0cHM6Ly9mb290aGlsbHRyYW5zaXQucmlkZXJhbGVydH...,aHR0cHM6Ly9mb290aGlsbHRyYW5zaXQucmlkZXJhbGVydH...,POINT (6569068.305 1827769.426)
8668,Foothill,major_stop_bus,1666,185,intersection_2_bus_routes_same_operator,Foothill,aHR0cHM6Ly9mb290aGlsbHRyYW5zaXQucmlkZXJhbGVydH...,aHR0cHM6Ly9mb290aGlsbHRyYW5zaXQucmlkZXJhbGVydH...,POINT (6572988.116 1826439.774)
8669,Foothill,major_stop_bus,1672,185,intersection_2_bus_routes_same_operator,Foothill,aHR0cHM6Ly9mb290aGlsbHRyYW5zaXQucmlkZXJhbGVydH...,aHR0cHM6Ly9mb290aGlsbHRyYW5zaXQucmlkZXJhbGVydH...,POINT (6571570.438 1821651.543)
8670,Foothill,major_stop_bus,1678,282,intersection_2_bus_routes_same_operator,Foothill,aHR0cHM6Ly9mb290aGlsbHRyYW5zaXQucmlkZXJhbGVydH...,aHR0cHM6Ly9mb290aGlsbHRyYW5zaXQucmlkZXJhbGVydH...,POINT (6571891.059 1822756.148)
8671,Foothill,major_stop_bus,1679,185,intersection_2_bus_routes_same_operator,Foothill,aHR0cHM6Ly9mb290aGlsbHRyYW5zaXQucmlkZXJhbGVydH...,aHR0cHM6Ly9mb290aGlsbHRyYW5zaXQucmlkZXJhbGVydH...,POINT (6571850.685 1822942.097)


In [13]:
stops_to_keep = df3.stop_id.unique().tolist()
from update_vars import COMPILED_CACHED_VIEWS, analysis_date


In [41]:
associated_feeds = [
    "1dce186c157f55ed353f9bd8bf6f43b6",
    "c7fac53170126479badfad5499f6048b",
    "ea4e5e79c2750d68d5d275df8a972ddd",
    "8a47f5aa51f481e9ddc7c497bd72d264",
    "415e0bd29d751e845b8b9dab3ba66b3e"
]

In [42]:
stop_times = pd.read_parquet(
    f"{COMPILED_CACHED_VIEWS}st_{analysis_date}.parquet", 
    filters = [[("stop_id", "in", stops_to_keep), 
               ("feed_key", "in", associated_feeds)]]
)

In [43]:
trips = pd.read_parquet(
    f"{COMPILED_CACHED_VIEWS}trips_{analysis_date}.parquet", 
    filters = [[("feed_key", "in", associated_feeds)]])

In [51]:
stop_times[(stop_times.stop_id=="1398") & 
           (stop_times.feed_key.isin(associated_feeds))].groupby(
    ["feed_key", "stop_id", "arrival_hour"]).agg(
    {"stop_sequence": "count"}).reset_index()

,feed_key,stop_id,arrival_hour,stop_sequence
0,8a47f5aa51f481e9ddc7c497bd72d264,1398,5,1
1,8a47f5aa51f481e9ddc7c497bd72d264,1398,6,2
2,8a47f5aa51f481e9ddc7c497bd72d264,1398,7,1
3,8a47f5aa51f481e9ddc7c497bd72d264,1398,8,1
4,8a47f5aa51f481e9ddc7c497bd72d264,1398,9,2
5,8a47f5aa51f481e9ddc7c497bd72d264,1398,10,1
6,8a47f5aa51f481e9ddc7c497bd72d264,1398,11,2
7,8a47f5aa51f481e9ddc7c497bd72d264,1398,12,1
8,8a47f5aa51f481e9ddc7c497bd72d264,1398,13,1
9,8a47f5aa51f481e9ddc7c497bd72d264,1398,14,2


In [40]:
feeds[feeds.name.str.contains("LADPW") & feeds.name.notna()]

,feed_key,name
28,415e0bd29d751e845b8b9dab3ba66b3e,LADPW Schedule


In [28]:
names = feeds[feeds.name.notna()].name.tolist()
names2 = [i.replace(' Schedule', '') for i in names]

In [36]:
agencies

['LA Metro', 'Foothill', 'Montebello', 'OCTA', 'LADPW']

In [ ]:
df2.explore("hqta_details", tiles='CartoDB Positron')